# Library Import #

In [26]:
import re
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os, glob
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
%matplotlib inline
plt.style.use('ggplot')

# Data Import

we'll use all of the dates up to the end of 2014 as our training data and everything after as testing data.

In [97]:
data = pd.read_csv('./input/Combined_News_DJIA.csv')
tweets = pd.read_csv('./input/realdonaldtrump.csv')
tweets["Date"] = pd.to_datetime(tweets["Date"])
tweets['Date']=tweets['Date'].dt.strftime('%Y-%m-%d')
tweets = tweets.groupby('Date')['Tweet'].apply(', '.join).reset_index()
tweets['Top26'] = tweets['Tweet'].str.replace('http\S+|www.\S+', ' ', case=False)
tweets.to_csv('./output/realdonaldtrump-cooked.csv')
tweets.drop(columns=["Tweet"], inplace=True)
data = pd.merge(tweets, data,how='left',on='Date')
data.drop(columns=["Label"], inplace=True)

#add more labels
more_labels = pd.read_csv('./input/DJIA_table_all_addlabel.csv')
# binary label
more_labels['Label']=np.where(more_labels['Open'] < more_labels['Close'], 1.0, 0.0)
# label_4part
#more_labels['Label'] = more_labels['Label_4part']
# label_3part
#more_labels['Label'] = more_labels['Label_3part']
binaryLabel = more_labels[['Date', 'Label', 'Label_4part', 'Label_3part']]
binaryLabel["Date"] = pd.to_datetime(binaryLabel["Date"])
binaryLabel['Date']=binaryLabel['Date'].dt.strftime('%Y-%m-%d')

data = pd.merge(data, binaryLabel,how='left',on='Date')

# Drop rows with any empty cells
data.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

#data.head(10)

#train = data[data['Date'] < '2015-05-31']
#test = data[data['Date'] > '2015-06-01']

<ipython-input-97-b541aefa82bb>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binaryLabel["Date"] = pd.to_datetime(binaryLabel["Date"])
<ipython-input-97-b541aefa82bb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binaryLabel['Date']=binaryLabel['Date'].dt.strftime('%Y-%m-%d')


,Date,Top26,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Label,Label_4part,Label_3part
0,2009-05-04,Be sure to tune in and watch Donald Trump on L...,b'Saudi Arabia to consider banning marriage of...,b'Pregnant woman miscarries after traffic acci...,"b'In four past pandemics, a mild spring outbre...",b'20 year old pregnant woman faces mandatory d...,"b""Without a Moral Leg to Stand On; United Arab...","b""Meet Noemi Letizia, the 18 year old hottie b...",b'So while everyone was fearing the Swine Flu ...,b'Defense Minister Ehud Barak said he would ca...,...,b'An interview with a jailed Somali pirate lea...,"b'Delara Darabi: ""Oh mother, I see the hangman...",b'SARS sleuth attacks WHO for being worthless ...,"b'Targeted by Taliban, Sikhs flee Pak region; ...","b'Israel Faces a Hard Sell: ""The American, Eur...",b'French Navy said they seized 11 pirates Sund...,"b'More Gaza patients quizzed at crossings, pre...",1.0,3.0,2.0
1,2009-05-08,Donald Trump reads Top Ten Financial Tips on L...,b'Eric Arthur Blair aka George Orwell must be...,b'British Muslim Immigrants More Strongly Iden...,"b""Irish student's fake quote on the Wikipedia ...","b""EU wants 'Internet G12' to govern cyberspace...","b""India's Election: how democracy may turn the...","b'Dr. Kamal Labwani, a prominent political and...","b'""We want to leave the city, but we cannot go...",b'Afghanistan: NATO Should Come Clean on White...,...,"b""1 million people now displaced by fighting i...",b'Sikh police in Britain push for bulletproof ...,b'Gunmen kill Darfur peacekeeper during carjac...,b'Saudi beauty queen: Veiled face and judged o...,b'Sierra Leone child miners: Legacy of conflict ',"b'CODEPINK Mothers Day 2009: ""I will not raise...","b'Gunman shoots three police, holed up in hous...",1.0,3.0,2.0
2,2009-05-12,"""My persona will never be that of a wallflower...",b'The PirateBay launches a D-DO$ attack. Wonde...,b'Israeli academics on the problems with Wikip...,"b""Change: China overtakes the US as Brazil's ...",b'Legalising Drugs: Lessons From Portugal',b'Video of a Guatemalan lawyer filmed the day ...,b'The World Health Organization is investigati...,"b'Cocaine Purity down, Prices up'",b'Guatemala: In YouTube Video Shot Before His ...,...,b'Sri Lankan government forces shells hospital...,"b""The man Netanyahu plans to make Israel's new...",b'Doctor says 49 killed in Sri Lanka hospital ...,b'49 killed in Sri Lanka hospital attack',"b""This is London the capital of Somali pirate...",b'Russia holds Victory Day celebration: rows o...,b'Torture: It Was Never Just Sleep Deprivation',1.0,2.0,2.0
3,2009-05-13,Listen to an interview with Donald Trump discu...,b'Swedish Bank to Freeze Accounts of The Pirat...,b'Guess who back in the TED house? Hans Roslin...,b'UK ISPs refuse to play Internet copyright cops',b'Pope demands independent Palestinian state: ...,b'Five men beheaded in Saudi Arabia after gros...,b'The Vatican made a surprise denial today tha...,"b""Sri Lanka has written off thousands of its o...",b'Guatemalan president accused of murder in vi...,...,"b""Roxana Saberi, the journalist recently relea...","b'Pope tells Palestinians: ""the wall that intr...",b'Kremlin: Battles over energy may lead to war...,b'MDC activist dies from ZANU-PF militia tortu...,b'Somalia: Islamist fighters poised to topple...,b'Data-Mining Predicted Swine Flu 18 Days befo...,"b""Scientists in Bolivia say that one of the co...",0.0,0.0,0.0
4,2009-05-14,"""Strive for wholeness and keep your sense of w...","b'""I was murdered by president Alvaro Colom""'","b'""Everyone is very angry with this wretched A...",b'Israel moves on plan to annex Palestinian la...,"b""Russia warns of war within a decade over Arc...","b'40,000-year-old figurine thought to be the o...",b'Gaza man: Israeli troops forced me to drink ...,b'Russia warns of war over Arctic resources',"b""Gordon Brown's government has repeatedly bla...",...,b'British Association for Adoption and Fosteri...,b

In [98]:
# add Trump tweet
tweets = pd.read_csv('./input/realdonaldtrump.csv')
tweets = tweets.groupby('Date')['Tweet'].apply(', '.join).reset_index()
#tweets.head(20)
#remove URLs in tweet
tweets["Date"] = pd.to_datetime(tweets["Date"])
tweets["Date"]=tweets["Date"].dt.strftime('%Y-%m-%d')
tweets['Tweet'] = tweets['Tweet'].str.replace('http\S+|www.\S+', ' ', case=False)
tweets['Tweet1'] = tweets['Tweet']
tweets.head(10)
tweets.to_csv('./output/realdonaldtrump-cooked.csv')

tweets = pd.merge(tweets, binaryLabel[['Date','Label', 'Label_3part','Label_4part']],how='left',on='Date')
# Drop rows with any empty cells
tweets.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

train = tweets[tweets['Date'] < '2019-05-31']
test = tweets[tweets['Date'] > '2019-06-01']

#train.head(10)

#tweets.drop(columns=["Tweet"], inplace=True)
#tweetsTrain = tweets[tweets['Date'] < '2015-05-31']
#tweetsTest = tweets[tweets['Date'] > '2015-06-01']

#train = pd.merge(train, tweetsTrain[['Date','Top26']],how='left',on='Date')
#test = pd.merge(test, tweetsTest[['Date','Top26']],how='left',on='Date')
#data.to_csv('./output/combined-tweets.csv')
#data.head(10)

,Date,Tweet,Tweet1,Label,Label_3part,Label_4part
7,2012-01-10,My @ FoxNews interview with @ gretawire where ...,My @ FoxNews interview with @ gretawire where ...,1.0,2.0,2.0
8,2013-01-10,"""Money may not grow from trees, but it does gr...","""Money may not grow from trees, but it does gr...",1.0,2.0,2.0
9,2014-01-10,""" @ Discoboy7 Salmond is on a one man crusade ...",""" @ Discoboy7 Salmond is on a one man crusade ...",0.0,1.0,1.0
12,2017-01-10,'U.S. Small-Business Optimism Index Surges by ...,'U.S. Small-Business Optimism Index Surges by ...,0.0,1.0,1.0
13,2018-01-10,It just shows everyone how broken and unfair o...,It just shows everyone how broken and unfair o...,1.0,1.0,2.0
14,2019-01-10,Cryin Chuck told his favorite lie when he used...,Cryin Chuck told his favorite lie when he used...,1.0,2.0,2.0
16,2012-01-11,"""President Reagan had it right: Social Securit...","""President Reagan had it right: Social Securit...",0.0,1.0,1.0
17,2013-01-11,"@Spot_O_TEA Thanks., @ Melbelle96 Thanks Melan...","@Spot_O_TEA Thanks., @ Melbelle96 Thanks Melan...",1.0,1.0,2.0
20,2016-01-11,Why does @ ThisWeekABC w/ @ GStephanopoulos al...,Why does @ ThisWeekABC w/ @ GStephanopoulos al...,1.0,1.0,2.0
21,2017-01-11,Russia just said the unverified report paid fo...,Russia just said the unverified report paid fo...,1.0,2.0,2.0


# Data Process

First, we transform the string of news into the  number of words as input.

In [29]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,1:2]))

In [30]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(2195, 31549)


## Logic Regression

### Logic Regression 1

Algorithm: Logic Regression

Input: the counts of single words

In [31]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

C:\Users\daih\Anaconda3\envs\st\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
basictest = basicvectorizer.transform(testheadlines)
preds1 = basicmodel.predict(basictest)
acc1=accuracy_score(test['Label'], preds1)

In [33]:
print('Logic Regression 1 accuracy: ',acc1 )

Logic Regression 1 accuracy:  0.3574144486692015


The accuracy is only 0.42.

Trump tweets as only source
0.51 accuracy if we use 2019/05/31 train/test data
0.49 accuracy if we use 2018/05/31 train/test data

In [34]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(5)

,Word,Coefficient
22486,questions,0.589787
6841,could,0.554146
12995,hope,0.508515
30626,wife,0.475421
30299,waste,0.467966


In [35]:
coeffdf.tail(5)

,Word,Coefficient
16687,lie,-0.514485
26876,success,-0.539553
18137,melania,-0.544801
7517,david,-0.577636
8201,did,-0.621151


### Logic Regression 2

Algorithm: Logic Regression

Input: the counts of phrases with two connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

We delete phrases of which frequency lower than 0.03 or higher than 0.97

In [36]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [37]:
print(advancedtrain.shape)

(2195, 412)


In [38]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [39]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)

In [40]:
print('Logic Regression 2 accuracy: ', acc2)

Logic Regression 2 accuracy:  0.3155893536121673


The accuracy is higher than input of single words.

In [41]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Words,Coefficient
303,they can,1.068292
83,going on,1.009603
121,in my,0.892559
324,to help,0.804999
333,to stop,0.799860


In [42]:
advcoeffdf.tail(5)

,Words,Coefficient
182,not the,-0.855420
407,you to,-0.898693
313,this morning,-0.907430
69,for our,-1.077052
27,as the,-1.125570


### Logic Regression 3

Algorithm: Logic Regression

Input: the counts of phrases with three connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

In [43]:
advancedvectorizer = TfidfVectorizer( min_df=0.0039, max_df=0.1, max_features = 200000, ngram_range = (3, 3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [44]:
print(advancedtrain.shape)

(2195, 1981)


In [45]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [46]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds3 = advancedmodel.predict(advancedtest)
acc3 = accuracy_score(test['Label'], preds3)

In [47]:
print('Logic Regression 3 accuracy: ', acc3)

Logic Regression 3 accuracy:  0.3726235741444867


The accuracy is lower than input of phrases with two connected words.

In [48]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Words,Coefficient
1208,should not be,1.060529
1234,squawkcnbc interview discussing,0.993034
1046,our leaders are,0.972492
1363,the first time,0.954336
1961,you think you,0.902798


In [49]:
advcoeffdf.tail(5)

,Words,Coefficient
1630,tomorrow night at,-0.695695
76,and their families,-0.698122
1357,the failing nytimes,-0.827777
1248,take care of,-0.859959
977,on behalf of,-0.882792


## Naive Bayes

### NBayes 1

In [50]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [51]:
print(advancedtrain.shape)

(2195, 242)


In [52]:
advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['Label'], preds4)

In [53]:
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.3269961977186312


In [54]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Words,Coefficient
240,you,-3.937314
161,realdonaldtrump,-4.008708
204,trump,-4.257856
101,it,-4.352573
80,great,-4.373338


In [55]:
advcoeffdf.tail(5)

,Words,Coefficient
145,off,-6.409500
194,things,-6.411220
172,smart,-6.452290
181,such,-6.478699
156,place,-6.527927


### NBayes 2

In [56]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [57]:
print(advancedtrain.shape)

(2195, 398)


In [58]:
advancedmodel = MultinomialNB(alpha=0.0001)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds5 = advancedmodel.predict(advancedtest)
acc5 = accuracy_score(test['Label'], preds5)

In [59]:
print('NBayes 2 accuracy: ', acc5)

NBayes 2 accuracy:  0.33840304182509506


In [60]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Words,Coefficient
68,for president,-4.954937
18,and the,-4.956402
145,last night,-5.011622
352,we are,-5.020182
197,our country,-5.024710


In [61]:
advcoeffdf.tail(5)

,Words,Coefficient
381,years ago,-6.922508
176,not the,-6.925774
263,the man,-6.939664
173,no one,-6.978273
223,season of,-6.981708


## Random Forest

### RF 1

In [62]:
advancedvectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [63]:
print(advancedtrain.shape)

(2195, 2144)


In [64]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds6 = advancedmodel.predict(advancedtest)
acc6 = accuracy_score(test['Label'], preds6)

In [65]:
print('RF 1 accuracy: ', acc6)

RF 1 accuracy:  0.30798479087452474


### RF 2

In [66]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [67]:
print(advancedtrain.shape)

(2195, 398)


In [68]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds7 = advancedmodel.predict(advancedtest)
acc7 = accuracy_score(test['Label'], preds7)

In [69]:
print('RF 2 accuracy: ', acc7)

RF 2 accuracy:  0.33840304182509506


## Gradient Boosting Machines

### GBM 1

In [70]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.9, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [71]:
print(advancedtrain.shape)

(2195, 250)


In [72]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds8 = advancedmodel.predict(advancedtest.toarray())
acc8 = accuracy_score(test['Label'], preds8)

In [73]:
print('GBM 1 accuracy: ', acc8)

GBM 1 accuracy:  0.34600760456273766


### GBM 2

In [74]:
advancedvectorizer = TfidfVectorizer( min_df=0.02, max_df=0.175, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [75]:
print(advancedtrain.shape)

(2195, 765)


In [76]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds9 = advancedmodel.predict(advancedtest.toarray())
acc9 = accuracy_score(test['Label'], preds9)

In [77]:
print('GBM 2 accuracy: ', acc9)

GBM 2 accuracy:  0.3574144486692015


## Stochastic Gradient Descent Classifier

### SGDClassifier 1

In [78]:
advancedvectorizer = TfidfVectorizer( min_df=0.2, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [79]:
print(advancedtrain.shape)

(2195, 108)


In [80]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds10 = advancedmodel.predict(advancedtest.toarray())
acc10 = accuracy_score(test['Label'], preds10)

TypeError: __init__() got an unexpected keyword argument 'n_iter'

In [ ]:
print('SGDClassifier 1: ', acc10)

### SGDClassifier 2

In [ ]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [ ]:
print(advancedtrain.shape)

In [ ]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds11 = advancedmodel.predict(advancedtest.toarray())
acc11 = accuracy_score(test['Label'], preds11)

In [ ]:
print('SGDClassifier 2: ', acc11)

## Naive Bayes SVM

In [ ]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):

    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

        #p_c /= np.linalg.norm(p_c, ord=1)
        #ratios[c] = np.log(p_c / (1 - p_c))


def f1_class(pred, truth, class_val):
    n = len(truth)

    truth_class = 0
    pred_class = 0
    tp = 0

    for ii in range(0, n):
        if truth[ii] == class_val:
            truth_class += 1
            if truth[ii] == pred[ii]:
                tp += 1
                pred_class += 1
                continue;
        if pred[ii] == class_val:
            pred_class += 1

    precision = tp / float(pred_class)
    recall = tp / float(truth_class)

    return (2.0 * precision * recall) / (precision + recall)


def semeval_senti_f1(pred, truth, pos=2, neg=0): 

    f1_pos = f1_class(pred, truth, pos)
    f1_neg = f1_class(pred, truth, neg)

    return (f1_pos + f1_neg) / 2.0;


def main(train_file, test_file, ngram=(1, 3)):
    print('loading...')
    train = pd.read_csv(train_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    # to shuffle:
    #train.iloc[np.random.permutation(len(df))]

    test = pd.read_csv(test_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    print('vectorizing...')
    vect = CountVectorizer()
    classifier = NBSVM()

    # create pipeline
    clf = Pipeline([('vect', vect), ('nbsvm', classifier)])
    params = {
        'vect__token_pattern': r"\S+",
        'vect__ngram_range': ngram, 
        'vect__binary': True
    }
    clf.set_params(**params)

    #X_train = vect.fit_transform(train['text'])
    #X_test = vect.transform(test['text'])

    print('fitting...')
    clf.fit(train['text'], train['label'])

    print('classifying...')
    pred = clf.predict(test['text'])
   
    print('testing...')
    acc = accuracy_score(test['label'], pred)
    f1 = semeval_senti_f1(pred, test['label'])
    print('NBSVM: acc=%f, f1=%f' % (acc, f1))

### NBSVM 1

In [ ]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

In [ ]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds12 = advancedmodel.predict(advancedtest)
acc12 = accuracy_score(test['Label'], preds12)

In [ ]:
print('NBSVM 1: ', acc12)

### NBSVM 2

In [ ]:
advancedvectorizer = TfidfVectorizer( min_df=0.031, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

In [ ]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds13 = advancedmodel.predict(advancedtest)
acc13 = accuracy_score(test['Label'], preds13)

In [ ]:
print('NBSVM 2: ', acc13)

## Deep Learning

### MLP

In [ ]:
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,1:2]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtrain.shape)

In [ ]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print("Training...")
model.fit(X_train, Y_train, nb_epoch=2, batch_size=16, validation_split=0.15, show_accuracy=True)

print("Generating test predictions...")
preds14 = model.predict_classes(X_test, verbose=0)
acc14 = accuracy_score(test["Label"], preds14)

In [ ]:
print('prediction accuracy: ', acc14)

### LSTM

In [ ]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2

In [ ]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

In [ ]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=3,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds15 = model.predict_classes(X_test, verbose=0)
acc15 = accuracy_score(test['Label'], preds15)

In [ ]:
print('prediction accuracy: ', acc15)

## CNN

In [ ]:
nb_filter = 120
filter_length = 2
hidden_dims = 120
nb_epoch = 2

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
print('Train...')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=1,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds16 = model.predict_classes(X_test, verbose=0)
acc16 = accuracy_score(test['Label'], preds16)

In [ ]:
print('prediction accuracy: ', acc16)

### BERT

#### Binary Classification

In [113]:
if __name__ == '__main__': 
    logging.basicConfig(level=logging.INFO)
    transformers_logger = logging.getLogger("transformers")
    transformers_logger.setLevel(logging.WARNING)

    # Preparing train data
    #train_data = [
    #    ["Aragorn was the heir of Isildur", 1],
    #    ["Frodo was the heir of Isildur", 0],
    #]
    #train_df = pd.DataFrame(train)
    #train_df.columns = ["Tweet", "label"]
    train_df = train[["Tweet", "Label"]]

    # Preparing eval data
    #eval_data = [
    #    ["Theoden was the king of Rohan", 1],
    #    ["Merry was the king of Rohan", 0],
    #]
    #eval_df = pd.DataFrame(test)
    #eval_df.columns = ["Tweet", "label"]
    eval_df = test[["Tweet", "Label"]]

    # Optional model configuration
    model_args = ClassificationArgs(num_train_epochs=2)

    model_args.overwrite_output_dir = True

    # Create a ClassificationModel
    model = ClassificationModel(
        "bert", "bert-base-cased", args=model_args
    )
    
    # Train the model
    model.train_model(train_df)

    # Evaluate the model
    #result, model_outputs, wrong_predictions = model.eval_model(eval_df)

    # Make predictions with the model
    X_test = test["Tweet"].values.tolist()
    Y_test = test["Label"].values.tolist()
    preds17, raw_outputs = model.predict(X_test)
    acc17 = accuracy_score(Y_test, preds17)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [117]:
print('prediction accuracy: ', acc17)

prediction accuracy:  0.5209125475285171


In [118]:
def GetMetrics(label,result):
    acc1 = accuracy_score(test['Label'], preds1)
    print('all all accuracy: ',acc1 )
    tmp = confusion_matrix(label, result)
    print(tmp)
    count = len(tmp[0])
    t1=[0,0,0,0]
    t2=[0,0,0,0]
    pre = [0,0,0,0]
    recall = [0,0,0,0]
    for i in range(count):
        for j in range(count):
            t1[i] += tmp[i][j]
            t2[j] += tmp[i][j]
    for i in range(count):
        recall[i] = round(tmp[i][i]/t1[i],4)
        pre[i] = round(tmp[i][i]/t2[i],4)
    print('pre: ', pre)
    print('recall: ', recall)

In [119]:
GetMetrics(test['Label'],preds17)

all all accuracy:  0.3193916349809886
[[ 14 109]
 [ 17 123]]
pre:  [0.4516, 0.5302, 0, 0]
recall:  [0.1138, 0.8786, 0, 0]


#### Multi-Classification

In [115]:
if __name__ == '__main__': 
    logging.basicConfig(level=logging.INFO)
    transformers_logger = logging.getLogger("transformers")
    transformers_logger.setLevel(logging.WARNING)

    # Preparing train data
    train_mf = train[["Tweet", "Label_4part"]]

    # Preparing eval data
    eval_mf = test[["Tweet", "Label_4part"]]

    # Optional model configuration
    model_args = ClassificationArgs(num_train_epochs=1)

    model_args.overwrite_output_dir = True

    # Create a ClassificationModel
    model = ClassificationModel(
        'bert',
        'bert-base-cased',
        num_labels=4,
        args=model_args,
        use_cuda=False
    ) 

    # Train the model
    model.train_model(train_df)

    # Evaluate the model
    #result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    
    # Make predictions with the model
    X_test = test["Tweet"].values.tolist()
    Y_test = test["Label_4part"].values.tolist()
    preds18, raw_outputs = model.predict(X_test)
    acc18 = accuracy_score(Y_test, preds18)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [122]:
print('prediction accuracy: ', acc18)

prediction accuracy:  0.3231939163498099


In [120]:
def GetMetrics(label,result):
    acc1 = accuracy_score(test['Label_4part'], preds1)
    print('all all accuracy: ',acc1 )
    tmp = confusion_matrix(label, result)
    print(tmp)
    count = len(tmp[0])
    t1=[0,0,0,0]
    t2=[0,0,0,0]
    pre = [0,0,0,0]
    recall = [0,0,0,0]
    for i in range(count):
        for j in range(count):
            t1[i] += tmp[i][j]
            t2[j] += tmp[i][j]
    for i in range(count):
        recall[i] = round(tmp[i][i]/t1[i],4)
        pre[i] = round(tmp[i][i]/t2[i],4)
    print('pre: ', pre)
    print('recall: ', recall)

In [121]:
GetMetrics(test['Label_4part'],preds18)

all all accuracy:  0.33079847908745247
[[  0  38   0   0]
 [  0  85   0   0]
 [  0 113   0   0]
 [  0  27   0   0]]
pre:  [nan, 0.3232, nan, nan]
recall:  [0.0, 1.0, 0.0, 0.0]


<ipython-input-120-e19f7a2df261>:17: RuntimeWarning: invalid value encountered in longlong_scalars
  pre[i] = round(tmp[i][i]/t2[i],4)
